<a href="https://colab.research.google.com/github/EricaCQ/data-structure-list-plain-algorithm/blob/main/Copy_of_Minera%C3%A7%C3%A3o_de_Textos_Trabalho_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Student: Mateus do Amor Devino Pereira**

Table of Contents:

* File Extraction and Listing
* Toy Example and Implementation
  * DF x Tokens
  * TF x Document
  * TF x IDF
  * Cosine Similarity
* Original Dataset
  * DF x Tokens
  * TF x Document
  * TF x IDF
  * Cosine Similarity
* Removing Stop Words
  * DF x Tokens
  * TF x Document
  * TF x IDF
  * Cosine Similarity



In [ ]:
from google.colab import files
from IPython.display import display, HTML

import math, nltk, os, string, sys, zipfile
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

ROOT_DIR = '/content'

print(f'Python version: {sys.version}')

Python version: 3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]


In [ ]:
# Downloading necessary nlkt packages
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# File Extraction and Listing

In [ ]:
# Upload sinopses.zip
f = files.upload()

Saving sinopses.zip to sinopses (3).zip


In [ ]:
# Extracting zip file
for filename, content in f.items():
  with zipfile.ZipFile(filename, 'r') as z:
    z.extractall(ROOT_DIR)

# Toy Example and Implementation

In [ ]:
# Toy example
filenames = ['d1', 'd2', 'd3', 'd4']
corpus = [
          'news about',
          'news about organic food campaign',
          'news of presidential campaign',
          'news of presidential campaign presidential candidate presidential election',
]
corpus = [nltk.tokenize.word_tokenize(document, language='portuguese') for document in corpus]
tokens = ['news', 'about', 'campaign', 'presidential', 'organic', 'food', 'of', 'candidate', 'election']

## DF x Tokens

In [ ]:
def df_matrix(tokens, documents):
  df = [[token in document for document in documents].count(True) for token in tokens]
  matrix = pd.DataFrame({'df': df}, tokens)
  matrix['idf'] = [math.log(len(documents) / matrix.loc[token, 'df'], 10) for token in matrix.index.values]
  
  return matrix

dfm = df_matrix(tokens, corpus)
display(HTML(dfm.to_html()))

,df,idf
news,4,0.000000
about,2,0.301030
campaign,3,0.124939
presidential,2,0.301030
organic,1,0.602060
food,1,0.602060
of,2,0.301030
candidate,1,0.602060
election,1,0.602060


## TF x Documents

In [ ]:
def tf_matrix(tokens, documents, document_names):
  aux = pd.DataFrame(np.zeros((len(documents), len(tokens))), filenames, tokens, int)
  
  # Counting token occurrences
  i = 0
  for document in documents:
    for word in document:
      aux.loc[document_names[i], word] += 1
    i += 1

  # Applying smoothing function
  matrix =  1 + np.log10(aux, where=aux > 0)
  matrix[aux == 0] *= 0
  return matrix

tfm = tf_matrix(tokens, corpus, filenames)
display(HTML(tfm.to_html()))

,news,about,campaign,presidential,organic,food,of,candidate,election
d1,1.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
d2,1.0,1.0,1.0,0.000000,1.0,1.0,0.0,0.0,0.0
d3,1.0,0.0,1.0,1.000000,0.0,0.0,1.0,0.0,0.0
d4,1.0,0.0,1.0,1.477121,0.0,0.0,1.0,1.0,1.0


## TF x IDF

In [ ]:
def tfidf_matrix(df_matrix, tf_matrix):
  return df_matrix['idf'] * tf_matrix

tfidfm = tfidf_matrix(dfm, tfm)
display(HTML(tfidfm.to_html()))

,news,about,campaign,presidential,organic,food,of,candidate,election
d1,0.0,0.30103,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000
d2,0.0,0.30103,0.124939,0.000000,0.60206,0.60206,0.00000,0.00000,0.00000
d3,0.0,0.00000,0.124939,0.301030,0.00000,0.00000,0.30103,0.00000,0.00000
d4,0.0,0.00000,0.124939,0.444658,0.00000,0.00000,0.30103,0.60206,0.60206


## Cosine Similarity

In [ ]:
csm = pd.DataFrame(cosine_similarity(tfidfm), filenames, filenames)
display(HTML(csm.to_html()))

,d1,d2,d3,d4
d1,1.000000,0.330188,0.000000,0.000000
d2,0.330188,1.000000,0.038591,0.016879
d3,0.000000,0.038591,1.000000,0.533471
d4,0.000000,0.016879,0.533471,1.000000


In [ ]:
def sort_similarity(df):
  # Selecting upper-diagonal, excluding main diagonal
  up = np.triu(np.ones(df.shape)).astype(np.bool)
  np.fill_diagonal(up, False)
  df = df.where(up)

  # Resetting indexes and sorting by similarity, from lowest to highest
  df = df.stack().reset_index()
  df.columns = ['Document A', 'Document B', 'Similarity']
  df.sort_values('Similarity', inplace=True, ascending=False)

  return df

similarities = sort_similarity(csm)
similarities.head(3)

,Document A,Document B,Similarity
5,d3,d4,0.533471
0,d1,d2,0.330188
3,d2,d3,0.038591


# Original Dataset

In [ ]:
# Listing extracted text files
filenames = os.listdir(os.path.join(ROOT_DIR, 'sinopses'))
filenames = [text_file for text_file in filenames if text_file.endswith('.txt')]
filenames.sort()

print('Text files found:', filenames)

# Reading files, lowering case and removing punctuation
translator = str.maketrans('', '', string.punctuation)
corpus = [open(os.path.join(ROOT_DIR, 'sinopses', filename), 'r').read().lower().translate(translator) for filename in filenames]

corpus = [nltk.tokenize.word_tokenize(document, language='portuguese') for document in corpus]
tokens = sorted(set([word for document in corpus for word in document]))

Text files found: ['1.txt', '10.txt', '2.txt', '3.txt', '4.txt', '5.txt', '6.txt', '7.txt', '8.txt', '9.txt']


## DF x Tokens

In [ ]:
dfm = df_matrix(tokens, corpus)
display(HTML(dfm.to_html()))

,df,idf
1827,2,0.698970
1923,1,1.000000
1949,1,1.000000
1972,2,0.698970
1980,1,1.000000
80,1,1.000000
a,8,0.096910
acontecimentos,1,1.000000
ada,1,1.000000
administrador,1,1.000000


## TF x Documents

In [ ]:
tfm = tf_matrix(tokens, corpus, filenames)
display(HTML(tfm.to_html()))

,1827,1923,1949,1972,1980,80,a,acontecimentos,ada,administrador,agora,aids,alcançar,alfredo,algumas,alguém,alterar,amada,amado,amante,amigo,amizade,amor,angeles,aniversário,anna,anos,antecederam,antigo,antigos,antiguidades,antipatiza,anton,ao,apaixonado,aprende,após,arranjado,ashton,assassino,assim,assistente,atraído,atriz,atuais,auditiva,aulas,autora,baines,bardem,baseada,beethoven,bela,bell,biografia,bolso,breve,brolin,brown,cada,cartas,casa,casamento,cascio,casou,caçador,certa,certo,chamado,chegada,chegam,chegar,cheia,chicago,chigurh,christopher,cidade,cineasta,cinema,ciumento,clarissa,clones,clássico,collier,colonizada,com,como,companhia,compositor,concorda,condições,conforme,conhece,conhecia,consegue,consequências,conseqüências,consertar,contaminado,contra,controle,coração,corpo,cresce,criança,criar,crise,cumprir,da,dalloway,daria,de,decide,deixandoo,deixava,delta,depressão,descobre,descobrir,desconhecido,desejo,deserto,desta,destinados,destino,detalhes,deu,devido,devolver,dias,diferentes,dinheiro,diz,dizer,do,dois,dolorosa,dona,dramaturgo,drogas,dá,década,e,echo,ed,editora,ela,ele,elise,em,empreende,encalço,encantado,encontrado,encontrando,encontrar,encontros,encontrála,enfrenta,enorme,enquanto,entanto,entidade,enviado,escapar,escritor,especificar,esperto,esquecer,esquiva,esta,estas,estava,estes,estreia,está,evan,ewan,existe,facilidade,falar,faleceu,fascinado,fatos,felix,festa,fica,ficar,ficou,filha,finalidade,fisicamente,flashbacks,flora,fora,forma,fornecer,foto,fotografia,futuro,garoto,gary,george,gera,gerando,gerente,grand,grande,grávida,gênio,habitantes,harris,harvey,hipnose,hipnotizado,histórico,hoje,holly,hospeda,hospedava,hotel,humanos,humor,hunter,há,idade,ido,idosa,idéias,ilha,imediatamente,imortal,infame,infância,iniciou,instrumento,intensidade,intensos,intimidade,intrigado,irritava,irá,island,isso,jacques,jane,javier,jeroen,johansson,jones,jordan,jornada,josh,jovem,julianne,juntamente,juntos,já,keitel,kidman,krabbe,krabbé,kutcher,laura,lee,lembrança,ler,lhe,ligadas,lincoln,livro,livros,lições,llewelyn,local,logo,los,ludwig,luta,lá,mackinac,maestro,maio,mais,mantêlos,marido,mas,mcgrath,mcgregor,mckenna,meio,meryl,mesma,mesmo,michigan,millfield,mim,mistério,moore,mora,morre,morrendo,morte,moss,motivo,mrs,muda,mudança,mulher,mulheres,musical,mínimos,na,narra,nas,negocia,neill,nem,nesta,nicole,no,noiret,noite,noivo,nome,nos,notícia,nova,novos,não,o,obsessão,oldman,onde,os,ou,pagamento,pagava,paquin,para,parar,parecem,partes,passa,passado,passar,pega,pelo,pequena,perda,perrin,períodos,peça,philippe,piano,piedade,planeja,planeta,plummer,pode,ponto,por,porém,possuem,possível,pouco,precisará,primeira,problemas,problemático,procura,procurálo,projecionista,proporcionada,próprio,psicótico,quando,que,reais,realidade,realiza,realizar,realmente,recebe,recriála,recusa,reeve,referida,região,regressão,relacionado,relógio,repleto,residentes,resolve,resolveu,retira,retrato,revela,richard,sabendo,sai,salvatore,salão,sam,scarlett,schindler,se,seis,sem,sendo,senhora,senso,separados,ser,seres,seu,seus,sexuais,seymour,sicília,situação,six,sonha,stewart,streep,sua,sucesso,suicídio,são,século,série,também,tanto,teatrólogo,teclas,televisão,tempo,tendo,tenta,tentando,tentar,ter,termina,terminar,testamento,texas,tinha,toda,todos,tom,tommy,tornam,toto,traficante,transformação,transportar,troca,trágicas,três,tudo,two,um,uma,universidade,utópica,vai,valise,van,vaughn,vez,viagem,viaja,viajar,vida,viena,virada,virginia,visita,vitoriana,vive,vivem,volta,volte,von,woolf,xerife,xx,york,zelândia,à,é,época,último,única,único
1.txt,0.0,0.0,0.0,1.0,1.0,0.0,1.30103,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.000000,0.0,0.0,0.00000,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0

## TF x IDF

In [ ]:
tfidfm = tfidf_matrix(dfm, tfm)
display(HTML(tfidfm.to_html()))

,1827,1923,1949,1972,1980,80,a,acontecimentos,ada,administrador,agora,aids,alcançar,alfredo,algumas,alguém,alterar,amada,amado,amante,amigo,amizade,amor,angeles,aniversário,anna,anos,antecederam,antigo,antigos,antiguidades,antipatiza,anton,ao,apaixonado,aprende,após,arranjado,ashton,assassino,assim,assistente,atraído,atriz,atuais,auditiva,aulas,autora,baines,bardem,baseada,beethoven,bela,bell,biografia,bolso,breve,brolin,brown,cada,cartas,casa,casamento,cascio,casou,caçador,certa,certo,chamado,chegada,chegam,chegar,cheia,chicago,chigurh,christopher,cidade,cineasta,cinema,ciumento,clarissa,clones,clássico,collier,colonizada,com,como,companhia,compositor,concorda,condições,conforme,conhece,conhecia,consegue,consequências,conseqüências,consertar,contaminado,contra,controle,coração,corpo,cresce,criança,criar,crise,cumprir,da,dalloway,daria,de,decide,deixandoo,deixava,delta,depressão,descobre,descobrir,desconhecido,desejo,deserto,desta,destinados,destino,detalhes,deu,devido,devolver,dias,diferentes,dinheiro,diz,dizer,do,dois,dolorosa,dona,dramaturgo,drogas,dá,década,e,echo,ed,editora,ela,ele,elise,em,empreende,encalço,encantado,encontrado,encontrando,encontrar,encontros,encontrála,enfrenta,enorme,enquanto,entanto,entidade,enviado,escapar,escritor,especificar,esperto,esquecer,esquiva,esta,estas,estava,estes,estreia,está,evan,ewan,existe,facilidade,falar,faleceu,fascinado,fatos,felix,festa,fica,ficar,ficou,filha,finalidade,fisicamente,flashbacks,flora,fora,forma,fornecer,foto,fotografia,futuro,garoto,gary,george,gera,gerando,gerente,grand,grande,grávida,gênio,habitantes,harris,harvey,hipnose,hipnotizado,histórico,hoje,holly,hospeda,hospedava,hotel,humanos,humor,hunter,há,idade,ido,idosa,idéias,ilha,imediatamente,imortal,infame,infância,iniciou,instrumento,intensidade,intensos,intimidade,intrigado,irritava,irá,island,isso,jacques,jane,javier,jeroen,johansson,jones,jordan,jornada,josh,jovem,julianne,juntamente,juntos,já,keitel,kidman,krabbe,krabbé,kutcher,laura,lee,lembrança,ler,lhe,ligadas,lincoln,livro,livros,lições,llewelyn,local,logo,los,ludwig,luta,lá,mackinac,maestro,maio,mais,mantêlos,marido,mas,mcgrath,mcgregor,mckenna,meio,meryl,mesma,mesmo,michigan,millfield,mim,mistério,moore,mora,morre,morrendo,morte,moss,motivo,mrs,muda,mudança,mulher,mulheres,musical,mínimos,na,narra,nas,negocia,neill,nem,nesta,nicole,no,noiret,noite,noivo,nome,nos,notícia,nova,novos,não,o,obsessão,oldman,onde,os,ou,pagamento,pagava,paquin,para,parar,parecem,partes,passa,passado,passar,pega,pelo,pequena,perda,perrin,períodos,peça,philippe,piano,piedade,planeja,planeta,plummer,pode,ponto,por,porém,possuem,possível,pouco,precisará,primeira,problemas,problemático,procura,procurálo,projecionista,proporcionada,próprio,psicótico,quando,que,reais,realidade,realiza,realizar,realmente,recebe,recriála,recusa,reeve,referida,região,regressão,relacionado,relógio,repleto,residentes,resolve,resolveu,retira,retrato,revela,richard,sabendo,sai,salvatore,salão,sam,scarlett,schindler,se,seis,sem,sendo,senhora,senso,separados,ser,seres,seu,seus,sexuais,seymour,sicília,situação,six,sonha,stewart,streep,sua,sucesso,suicídio,são,século,série,também,tanto,teatrólogo,teclas,televisão,tempo,tendo,tenta,tentando,tentar,ter,termina,terminar,testamento,texas,tinha,toda,todos,tom,tommy,tornam,toto,traficante,transformação,transportar,troca,trágicas,três,tudo,two,um,uma,universidade,utópica,vai,valise,van,vaughn,vez,viagem,viaja,viajar,vida,viena,virada,virginia,visita,vitoriana,vive,vivem,volta,volte,von,woolf,xerife,xx,york,zelândia,à,é,época,último,única,único
1.txt,0.00000,0.0,0.0,0.69897,1.0,0.0,0.126083,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,1.0,0.0,1.0,0.0,0.00000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.698970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.69897,0.0,0.221849,0.00000,0.

# Cosine Similarity

In [ ]:
csm = pd.DataFrame(cosine_similarity(tfidfm), filenames, filenames)
display(HTML(csm.to_html()))

,1.txt,10.txt,2.txt,3.txt,4.txt,5.txt,6.txt,7.txt,8.txt,9.txt
1.txt,1.000000,0.012879,0.016516,0.017078,0.011118,0.151159,0.020753,0.013116,0.050469,0.035237
10.txt,0.012879,1.000000,0.008390,0.023075,0.017697,0.010419,0.020678,0.001646,0.028841,0.011217
2.txt,0.016516,0.008390,1.000000,0.041091,0.027932,0.025791,0.014302,0.043977,0.009884,0.028360
3.txt,0.017078,0.023075,0.041091,1.000000,0.000551,0.011201,0.008465,0.025861,0.035322,0.026500
4.txt,0.011118,0.017697,0.027932,0.000551,1.000000,0.012570,0.001405,0.000873,0.043494,0.007126
5.txt,0.151159,0.010419,0.025791,0.011201,0.012570,1.000000,0.016463,0.014681,0.027253,0.011817
6.txt,0.020753,0.020678,0.014302,0.008465,0.001405,0.016463,1.000000,0.184118,0.007366,0.002371
7.txt,0.013116,0.001646,0.043977,0.025861,0.000873,0.014681,0.184118,1.000000,0.015375,0.002810
8.txt,0.050469,0.028841,0.009884,0.035322,0.043494,0.027253,0.007366,0.015375,1.000000,0.013784
9.txt,0.035237,0.011217,0.028360,0.026500,0.007126,0.011817,0.002371,0.002810,0.013784,1.000000


In [ ]:
similarities = sort_similarity(csm)
similarities.head(3)

,Document A,Document B,Similarity
39,6.txt,7.txt,0.184118
4,1.txt,5.txt,0.151159
7,1.txt,8.txt,0.050469


# Removing Stop Words

In [ ]:
# Listing extracted text files
filenames = os.listdir(os.path.join(ROOT_DIR, 'sinopses'))
filenames = [text_file for text_file in filenames if text_file.endswith('.txt')]
filenames.sort()

print('Text files found:', filenames)

# Reading files, lowering case and removing punctuation
translator = str.maketrans('', '', string.punctuation)
corpus = [open(os.path.join(ROOT_DIR, 'sinopses', filename), 'r').read().lower().translate(translator) for filename in filenames]
corpus = [nltk.tokenize.word_tokenize(document, language='portuguese') for document in corpus]

# Removing stopworks from tokens list
stopwords = nltk.corpus.stopwords.words('portuguese')
corpus = [[word for word in document if word not in stopwords] for document in corpus]
tokens = sorted(set([word for document in corpus for word in document]))

Text files found: ['1.txt', '10.txt', '2.txt', '3.txt', '4.txt', '5.txt', '6.txt', '7.txt', '8.txt', '9.txt']


## DF x Tokens

In [ ]:
dfm = df_matrix(tokens, corpus)
display(HTML(dfm.to_html()))

,df,idf
1827,2,0.698970
1923,1,1.000000
1949,1,1.000000
1972,2,0.698970
1980,1,1.000000
80,1,1.000000
acontecimentos,1,1.000000
ada,1,1.000000
administrador,1,1.000000
agora,1,1.000000


## TF x Documents

In [ ]:
tfm = tf_matrix(tokens, corpus, filenames)
display(HTML(tfm.to_html()))

,1827,1923,1949,1972,1980,80,acontecimentos,ada,administrador,agora,aids,alcançar,alfredo,algumas,alguém,alterar,amada,amado,amante,amigo,amizade,amor,angeles,aniversário,anna,anos,antecederam,antigo,antigos,antiguidades,antipatiza,anton,apaixonado,aprende,após,arranjado,ashton,assassino,assim,assistente,atraído,atriz,atuais,auditiva,aulas,autora,baines,bardem,baseada,beethoven,bela,bell,biografia,bolso,breve,brolin,brown,cada,cartas,casa,casamento,cascio,casou,caçador,certa,certo,chamado,chegada,chegam,chegar,cheia,chicago,chigurh,christopher,cidade,cineasta,cinema,ciumento,clarissa,clones,clássico,collier,colonizada,companhia,compositor,concorda,condições,conforme,conhece,conhecia,consegue,consequências,conseqüências,consertar,contaminado,contra,controle,coração,corpo,cresce,criança,criar,crise,cumprir,dalloway,daria,decide,deixandoo,deixava,delta,depressão,descobre,descobrir,desconhecido,desejo,deserto,desta,destinados,destino,detalhes,deu,devido,devolver,dias,diferentes,dinheiro,diz,dizer,dois,dolorosa,dona,dramaturgo,drogas,dá,década,echo,ed,editora,elise,empreende,encalço,encantado,encontrado,encontrando,encontrar,encontros,encontrála,enfrenta,enorme,enquanto,entanto,entidade,enviado,escapar,escritor,especificar,esperto,esquecer,esquiva,estreia,evan,ewan,existe,facilidade,falar,faleceu,fascinado,fatos,felix,festa,fica,ficar,ficou,filha,finalidade,fisicamente,flashbacks,flora,forma,fornecer,foto,fotografia,futuro,garoto,gary,george,gera,gerando,gerente,grand,grande,grávida,gênio,habitantes,harris,harvey,hipnose,hipnotizado,histórico,hoje,holly,hospeda,hospedava,hotel,humanos,humor,hunter,idade,ido,idosa,idéias,ilha,imediatamente,imortal,infame,infância,iniciou,instrumento,intensidade,intensos,intimidade,intrigado,irritava,irá,island,jacques,jane,javier,jeroen,johansson,jones,jordan,jornada,josh,jovem,julianne,juntamente,juntos,keitel,kidman,krabbe,krabbé,kutcher,laura,lee,lembrança,ler,ligadas,lincoln,livro,livros,lições,llewelyn,local,logo,los,ludwig,luta,lá,mackinac,maestro,maio,mantêlos,marido,mcgrath,mcgregor,mckenna,meio,meryl,mesma,michigan,millfield,mim,mistério,moore,mora,morre,morrendo,morte,moss,motivo,mrs,muda,mudança,mulher,mulheres,musical,mínimos,narra,negocia,neill,nesta,nicole,noiret,noite,noivo,nome,notícia,nova,novos,obsessão,oldman,onde,pagamento,pagava,paquin,parar,parecem,partes,passa,passado,passar,pega,pequena,perda,perrin,períodos,peça,philippe,piano,piedade,planeja,planeta,plummer,pode,ponto,porém,possuem,possível,pouco,precisará,primeira,problemas,problemático,procura,procurálo,projecionista,proporcionada,próprio,psicótico,reais,realidade,realiza,realizar,realmente,recebe,recriála,recusa,reeve,referida,região,regressão,relacionado,relógio,repleto,residentes,resolve,resolveu,retira,retrato,revela,richard,sabendo,sai,salvatore,salão,sam,scarlett,schindler,seis,sendo,senhora,senso,separados,ser,seres,sexuais,seymour,sicília,situação,six,sonha,stewart,streep,sucesso,suicídio,século,série,tanto,teatrólogo,teclas,televisão,tempo,tendo,tenta,tentando,tentar,ter,termina,terminar,testamento,texas,toda,todos,tom,tommy,tornam,toto,traficante,transformação,transportar,troca,trágicas,três,tudo,two,universidade,utópica,vai,valise,van,vaughn,vez,viagem,viaja,viajar,vida,viena,virada,virginia,visita,vitoriana,vive,vivem,volta,volte,von,woolf,xerife,xx,york,zelândia,época,último,única,único
1.txt,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.00000,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

## TF x IDF

In [ ]:
tfidfm = tfidf_matrix(dfm, tfm)
display(HTML(tfidfm.to_html()))

,1827,1923,1949,1972,1980,80,acontecimentos,ada,administrador,agora,aids,alcançar,alfredo,algumas,alguém,alterar,amada,amado,amante,amigo,amizade,amor,angeles,aniversário,anna,anos,antecederam,antigo,antigos,antiguidades,antipatiza,anton,apaixonado,aprende,após,arranjado,ashton,assassino,assim,assistente,atraído,atriz,atuais,auditiva,aulas,autora,baines,bardem,baseada,beethoven,bela,bell,biografia,bolso,breve,brolin,brown,cada,cartas,casa,casamento,cascio,casou,caçador,certa,certo,chamado,chegada,chegam,chegar,cheia,chicago,chigurh,christopher,cidade,cineasta,cinema,ciumento,clarissa,clones,clássico,collier,colonizada,companhia,compositor,concorda,condições,conforme,conhece,conhecia,consegue,consequências,conseqüências,consertar,contaminado,contra,controle,coração,corpo,cresce,criança,criar,crise,cumprir,dalloway,daria,decide,deixandoo,deixava,delta,depressão,descobre,descobrir,desconhecido,desejo,deserto,desta,destinados,destino,detalhes,deu,devido,devolver,dias,diferentes,dinheiro,diz,dizer,dois,dolorosa,dona,dramaturgo,drogas,dá,década,echo,ed,editora,elise,empreende,encalço,encantado,encontrado,encontrando,encontrar,encontros,encontrála,enfrenta,enorme,enquanto,entanto,entidade,enviado,escapar,escritor,especificar,esperto,esquecer,esquiva,estreia,evan,ewan,existe,facilidade,falar,faleceu,fascinado,fatos,felix,festa,fica,ficar,ficou,filha,finalidade,fisicamente,flashbacks,flora,forma,fornecer,foto,fotografia,futuro,garoto,gary,george,gera,gerando,gerente,grand,grande,grávida,gênio,habitantes,harris,harvey,hipnose,hipnotizado,histórico,hoje,holly,hospeda,hospedava,hotel,humanos,humor,hunter,idade,ido,idosa,idéias,ilha,imediatamente,imortal,infame,infância,iniciou,instrumento,intensidade,intensos,intimidade,intrigado,irritava,irá,island,jacques,jane,javier,jeroen,johansson,jones,jordan,jornada,josh,jovem,julianne,juntamente,juntos,keitel,kidman,krabbe,krabbé,kutcher,laura,lee,lembrança,ler,ligadas,lincoln,livro,livros,lições,llewelyn,local,logo,los,ludwig,luta,lá,mackinac,maestro,maio,mantêlos,marido,mcgrath,mcgregor,mckenna,meio,meryl,mesma,michigan,millfield,mim,mistério,moore,mora,morre,morrendo,morte,moss,motivo,mrs,muda,mudança,mulher,mulheres,musical,mínimos,narra,negocia,neill,nesta,nicole,noiret,noite,noivo,nome,notícia,nova,novos,obsessão,oldman,onde,pagamento,pagava,paquin,parar,parecem,partes,passa,passado,passar,pega,pequena,perda,perrin,períodos,peça,philippe,piano,piedade,planeja,planeta,plummer,pode,ponto,porém,possuem,possível,pouco,precisará,primeira,problemas,problemático,procura,procurálo,projecionista,proporcionada,próprio,psicótico,reais,realidade,realiza,realizar,realmente,recebe,recriála,recusa,reeve,referida,região,regressão,relacionado,relógio,repleto,residentes,resolve,resolveu,retira,retrato,revela,richard,sabendo,sai,salvatore,salão,sam,scarlett,schindler,seis,sendo,senhora,senso,separados,ser,seres,sexuais,seymour,sicília,situação,six,sonha,stewart,streep,sucesso,suicídio,século,série,tanto,teatrólogo,teclas,televisão,tempo,tendo,tenta,tentando,tentar,ter,termina,terminar,testamento,texas,toda,todos,tom,tommy,tornam,toto,traficante,transformação,transportar,troca,trágicas,três,tudo,two,universidade,utópica,vai,valise,van,vaughn,vez,viagem,viaja,viajar,vida,viena,virada,virginia,visita,vitoriana,vive,vivem,volta,volte,von,woolf,xerife,xx,york,zelândia,época,último,única,único
1.txt,0.00000,0.0,0.0,0.69897,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,1.0,0.0,1.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.698970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.69897,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.69897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.39794,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.69897,0.0,0.0,0.00000

## Cosine Similarity

In [ ]:
csm = pd.DataFrame(cosine_similarity(tfidfm), filenames, filenames)
display(HTML(csm.to_html()))

,1.txt,10.txt,2.txt,3.txt,4.txt,5.txt,6.txt,7.txt,8.txt,9.txt
1.txt,1.000000,0.000000,0.003293,0.011047,0.000000,0.147963,0.008150,0.010452,0.016576,0.030167
10.txt,0.000000,1.000000,0.005922,0.007287,0.006543,0.000000,0.013097,0.000000,0.020229,0.009558
2.txt,0.003293,0.005922,1.000000,0.029627,0.021297,0.010892,0.011331,0.030870,0.000000,0.014991
3.txt,0.011047,0.007287,0.029627,1.000000,0.000000,0.000000,0.005113,0.023144,0.014847,0.011513
4.txt,0.000000,0.006543,0.021297,0.000000,1.000000,0.000000,0.000000,0.000000,0.008548,0.000000
5.txt,0.147963,0.000000,0.010892,0.000000,0.000000,1.000000,0.013479,0.012679,0.010582,0.008131
6.txt,0.008150,0.013097,0.011331,0.005113,0.000000,0.013479,1.000000,0.185185,0.003103,0.000000
7.txt,0.010452,0.000000,0.030870,0.023144,0.000000,0.012679,0.185185,1.000000,0.002919,0.000000
8.txt,0.016576,0.020229,0.000000,0.014847,0.008548,0.010582,0.003103,0.002919,1.000000,0.010898
9.txt,0.030167,0.009558,0.014991,0.011513,0.000000,0.008131,0.000000,0.000000,0.010898,1.000000


In [ ]:
similarities = sort_similarity(csm)
similarities.head(3)

,Document A,Document B,Similarity
39,6.txt,7.txt,0.185185
4,1.txt,5.txt,0.147963
21,2.txt,7.txt,0.030870
